Laboratorio 11
Ejercicio 1

In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder

# Load the dataset
file_path = 'Jugadores.xlsx'
df = pd.read_excel(file_path)

# Display the first few rows of the dataset
print(df.head())

# Preprocess the data
# Check for missing values
print(df.isnull().sum())

# Handle missing values if any (you can choose to drop or fill them)
# For this example, we'll drop rows with missing values
df.dropna(inplace=True)

# Identify columns with string (object) dtype
object_columns = df.select_dtypes(include=['object']).columns
print("Object columns:", object_columns) # Print these columns to inspect them

# Decide how to handle these columns:
# 1. Drop them if they are not relevant
# 2. Encode them if they are categorical
# 3. Convert them to numerical if possible (e.g., extracting numbers from strings)

# Example: Dropping object columns (replace this with your chosen method)
df = df.drop(object_columns, axis=1)

# Encode the categorical variable 'Posicion Num' (assuming it is categorical)
le = LabelEncoder()
df['Posicion Num'] = le.fit_transform(df['Posicion Num'])

# Separate features and labels
X = df.drop('Label', axis=1)
y = df['Label']

# Normalize numerical features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X) # Now this should work without errors

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

# Create the Sequential model
model = Sequential([
    Dense(64, activation='relu', input_shape=(X_train.shape[1],)),
    Dense(32, activation='relu'),
    Dense(1, activation='sigmoid')  # Assuming binary classification for 'Label'
])

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
history = model.fit(X_train, y_train, epochs=50, batch_size=32, validation_split=0.2)

# Evaluate the model on the test set
test_loss, test_accuracy = model.evaluate(X_test, y_test)
print(f'Test accuracy: {test_accuracy:.4f}')

# Save the model to a .h5 file
model.save('player_scouting_model.h5')



   Numero de Jugador            Jugador Pais Posicion  Posicion Num  Edad  \
0           0.797908  Cristiano Ronaldo  POR     DLCC            43    29   
1           0.614087              Djené  TOG       DF             2    29   
2           0.514732            Rodrigo  ESP       DL             4    21   
3           0.866826         Allan Nyom  CMR       DF             2    33   
4           0.839183   Javier Ontiveros  ESP     CCDL            34    23   

   Partidos Jugados  Titular  Minutos  Goles  ...  Goles Sin Penalizacion  \
0                35       35     3100     48  ...                      38   
1                35       35     3115      0  ...                       0   
2                21       10      881      1  ...                       1   
3                 5        3      244      0  ...                       0   
4                 3        0       39      0  ...                       0   

   Penales Acertados  Total Penales  Tarjetas Amarillas  Tarjetas Rojas  \

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


Ejercicio 2

In [ ]:
import numpy as np
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import load_model

# Cargar el modelo preentrenado
modelo = load_model('modelo_jugadores.h5')

# Datos de los jugadores para predecir
jugadores = {
    'Harry Maguire': [2, 28, 30, 1, 0, 0, 9, 1, 7],
    'Karim Benzema': [4, 33, 32, 27, 12, 7, 0, 0, 5],
    'Robert Lewandowski': [4, 32, 34, 35, 3, 5, 2, 0, 32],
    'Ximo Navarro': [2, 31, 18, 0, 0, 0, 4, 1, 0.5]
}

# Convertir los datos de los jugadores en un array
jugadores_array = np.array(list(jugadores.values()))

# Normalizar características
scaler = StandardScaler()
jugadores_scaled = scaler.fit_transform(jugadores_array)

# Realizar predicciones
predicciones = modelo.predict(jugadores_scaled)

# Analizar resultados
for jugador, prediccion in zip(jugadores.keys(), predicciones):
    print(f"El jugador {jugador} tiene una probabilidad de ser titular del {prediccion[0]*100:.2f}%.")


1/1 [==============================] - 1s 948ms/step
El jugador Harry Maguire tiene una probabilidad de ser titular del 37.85%.
El jugador Karim Benzema tiene una probabilidad de ser titular del 95.59%.
El jugador Robert Lewandowski tiene una probabilidad de ser titular del 12.28%.
El jugador Ximo Navarro tiene una probabilidad de ser titular del 73.54%.


Ejercicio 3

In [ ]:
import tensorflow as tf
from tensorflow.keras.datasets import mnist
from tensorflow.keras.utils import to_categorical

# Load the MNIST dataset
(x_train, y_train), (x_test, y_test) = mnist.load_data()

# Reshape the data to fit the model
x_train = x_train.reshape(x_train.shape[0], 28, 28, 1)
x_test = x_test.reshape(x_test.shape[0], 28, 28, 1)

# Normalize the data
x_train = x_train.astype('float32') / 255
x_test = x_test.astype('float32') / 255

# One-hot encode the labels
y_train = to_categorical(y_train, 10)
y_test = to_categorical(y_test, 10)

# Create the model
model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(28, 28, 1)),
    tf.keras.layers.Conv2D(64, kernel_size=(3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D(pool_size=(2, 2)),
    tf.keras.layers.Dropout(0.25),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(10, activation='softmax')
])

# Compile the model
model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

# Train the model
history = model.fit(
    x_train, y_train,
    epochs=12,
    batch_size=32,
    validation_data=(x_test, y_test)
)

# Save the model to an .h5 file
model.save('mnist_cnn_model.h5')

print("Model training complete and saved as 'mnist_cnn_model.h5'")


11490434/11490434 [==============================] - 0s 0us/step
Epoch 1/12
1875/1875 [==============================] - 28s 8ms/step - loss: 0.1929 - accuracy: 0.9427 - val_loss: 0.0472 - val_accuracy: 0.9838
Epoch 2/12
1875/1875 [==============================] - 9s 5ms/step - loss: 0.0800 - accuracy: 0.9757 - val_loss: 0.0369 - val_accuracy: 0.9877
Epoch 3/12
1875/1875 [==============================] - 11s 6ms/step - loss: 0.0606 - accuracy: 0.9818 - val_loss: 0.0333 - val_accuracy: 0.9892
Epoch 4/12
1875/1875 [==============================] - 11s 6ms/step - loss: 0.0507 - accuracy: 0.9846 - val_loss: 0.0352 - val_accuracy: 0.9884
Epoch 5/12
1875/1875 [==============================] - 10s 5ms/step - loss: 0.0421 - accuracy: 0.9874 - val_loss: 0.0300 - val_accuracy: 0.9910
Epoch 6/12
1875/1875 [==============================] - 9s 5ms/step - loss: 0.0365 - accuracy: 0.9885 - val_loss: 0.0292 - val_accuracy: 0.9915
Epoch 7/12
1875/1875 [==============================] - 10s 5ms/ste

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


Ejercicio 4

In [ ]:
import tensorflow as tf
from tensorflow.keras.datasets import mnist
from tensorflow.keras.models import load_model
import numpy as np

# Load the MNIST dataset
(x_train, y_train), (x_test, y_test) = mnist.load_data()

# Reshape the test data to fit the model
x_test = x_test.reshape(x_test.shape[0], 28, 28, 1)

# Normalize the test data
x_test = x_test.astype('float32') / 255

# Load the pre-trained model
model = load_model('mnist_cnn_model.h5')

# Function to make a prediction on a single image
def predict_single_image(model, image, true_label):
    # Expand dimensions to fit the model input
    image = np.expand_dims(image, axis=0)
    # Make the prediction
    prediction = model.predict(image)
    # Get the predicted label
    predicted_label = np.argmax(prediction)
    print(f'Expected: {true_label}, Predicted: {predicted_label}')

# Test the model with specific images
# x_test[1] is the number 2
predict_single_image(model, x_test[1], 2)

# x_test[12] is the number 9
predict_single_image(model, x_test[12], 9)

# x_test[200] is the number 3
predict_single_image(model, x_test[200], 3)


1/1 [==============================] - 0s 203ms/step
Expected: 2, Predicted: 2
1/1 [==============================] - 0s 22ms/step
Expected: 9, Predicted: 9
1/1 [==============================] - 0s 18ms/step
Expected: 3, Predicted: 3
